In [1]:
import sys
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages
import surprise

from recommenders.utils.general_utils import get_number_processors
from recommenders.utils.gpu_utils import get_cuda_version, get_cudnn_version
from recommenders.datasets.python_splitters import python_stratified_split

from sklearn.preprocessing import LabelEncoder

from benchmark_utils import * 

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Surprise version: {}".format(surprise.__version__))
print("PyTorch version: {}".format(torch.__version__))
print("Cornac version: {}".format(cornac.__version__))
print("Tensorflow version: {}".format(tf.__version__))
print("CUDA version: {}".format(get_cuda_version()))
print("CuDNN version: {}".format(get_cudnn_version()))
n_cores = get_number_processors()
print("Number of cores: {}".format(n_cores))

%load_ext autoreload
%autoreload 2

System version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
Pandas version: 1.3.4
Surprise version: 1.1.1
PyTorch version: 1.10.1+cu102
Cornac version: 1.14.1
Tensorflow version: 2.7.0
CUDA version: No CUDA in this machine
CuDNN version: No CUDNN in this machine
Number of cores: 48


In [2]:
# fix random seeds to make sure out runs are reproducible
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [3]:
environments = {
    "svd": "python_cpu",
    "ncf": "python_gpu",
    "bpr": "python_cpu",
    "bivae": "python_gpu",
}

metrics = {
    "svd": ["ranking"],
    "ncf": ["ranking"],
    "bpr": ["ranking"],
    "bivae": ["ranking"],
}

In [4]:
svd_params = {
    "n_factors": 150,
    "n_epochs": 15,
    "lr_all": 0.005,
    "reg_all": 0.02,
    "random_state": SEED,
    "verbose": False
}

ncf_params = {
    "model_type": "NeuMF",
    "n_factors": 4,
    "layer_sizes": [16, 8, 4],
    "n_epochs": 15,
    "batch_size": 1024,
    "learning_rate": 1e-3,
    "verbose": 10
}

bpr_params = {
    "k": 200,
    "max_iter": 200,
    "learning_rate": 0.01,
    "lambda_reg": 1e-3,
    "seed": SEED,
    "verbose": False
}

bivae_params = {
    "k": 100,
    "encoder_structure": [200],
    "act_fn": "tanh",
    "likelihood": "pois",
    "n_epochs": 500,
    "batch_size": 1024,
    "learning_rate": 0.001,
    "seed": SEED,
    "use_gpu": True,
    "verbose": False
}

params = {
    "svd": svd_params,
    "ncf": ncf_params,
    "bpr": bpr_params,
    "bivae": bivae_params,
}

In [5]:
prepare_training_data = {
    "svd": prepare_training_svd,
    "ncf": prepare_training_ncf,
    "bpr": prepare_training_cornac,
    "bivae": prepare_training_cornac,
}

In [6]:
prepare_metrics_data = {
    "als": lambda train, test: prepare_metrics_als(train, test),
    "fastai": lambda train, test: prepare_metrics_fastai(train, test),    
}

In [7]:
trainer = {
    "svd": lambda params, data: train_svd(params, data),
    "ncf": lambda params, data: train_ncf(params, data),
    "bpr": lambda params, data: train_bpr(params, data),
    "bivae": lambda params, data: train_bivae(params, data),
}

In [8]:
ranking_predictor = {
    "svd": lambda model, test, train: recommend_k_svd(model, test, train),
    "ncf": lambda model, test, train: recommend_k_ncf(model, test, train),
    "bpr": lambda model, test, train: recommend_k_cornac(model, test, train),
    "bivae": lambda model, test, train: recommend_k_cornac(model, test, train),
}

ranking_evaluator = {
    "svd": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "ncf": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "bpr": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "bivae": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
}

def generate_summary(data, algo, k, train_time, time_ranking, ranking_metrics):
    summary = {"Data": data, "Algo": algo, "K": k, "Train time (s)": train_time, "Recommending time (s)": time_ranking}

    if ranking_metrics is None:
        ranking_metrics = {
            "MAP": np.nan,
            "nDCG@k": np.nan,
            "Precision@k": np.nan,
            "Recall@k": np.nan,
        }
    summary.update(ranking_metrics)
    return summary

In [9]:
algorithms = ['svd', 'bpr', "ncf", "bivae"]

In [10]:
# For each data size and each algorithm, a recommender is evaluated. 
cols = ["Data", "Algo", "K", "Train time (s)", "Recommending time (s)", "MAP", "nDCG@k", "Precision@k", "Recall@k"]
df_results = pd.DataFrame(columns=cols)

# Load the dataset
df = pd.read_csv('data/Vote.txt', sep='\t', header=None)
df.columns = ['userid', 'itemid', 'rating', 'weight', 'timestamp']
df.rating = df.rating * 5
df = df[['userid', 'itemid', 'rating', 'timestamp']]
df.columns = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL]
print("Size of EachMovie: {}".format(df.shape))

# Split the dataset
df_train, df_test = python_stratified_split(df,
                                            ratio=0.75, 
                                            min_rating=1, 
                                            filter_by="item", 
                                            col_user=DEFAULT_USER_COL, 
                                            col_item=DEFAULT_ITEM_COL
                                            )

# Loop through the algos
for algo in algorithms:
    print(algo)
    # Data prep for training set
    train = prepare_training_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
    
    # Get model parameters
    model_params = params[algo]
      
    # Train the model
    model, time_train = trainer[algo](model_params, train)
            
    # Predict and evaluate
    train, test = prepare_metrics_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
    
    if "ranking" in metrics[algo]:
        # Predict for ranking
        top_k_scores, time_ranking = ranking_predictor[algo](model, test, train)
        
        # Evaluate for rating
        for k in [10, 50, 100]:
            
            DEFAULT_K = k
            rankings = ranking_evaluator[algo](test, top_k_scores, DEFAULT_K)
            
            # Record results
            summary = generate_summary('EachMovie', algo, DEFAULT_K, time_train, time_ranking, rankings)
            df_results.loc[df_results.shape[0] + 1] = summary
    
print("\nComputation finished")

Size of EachMovie: (2811983, 4)
svd
bpr


/dev/shm/mert.burabak/.conda/envs/mert_venv/lib/python3.7/site-packages/cornac/data/dataset.py:361: UserWarning: 141 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


ncf


/dev/shm/mert.burabak/.conda/envs/mert_venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


bivae

Computation finished


In [11]:
df_results

,Data,Algo,K,Train time (s),Recommending time (s),MAP,nDCG@k,Precision@k,Recall@k
1,EachMovie,svd,10,128.9592,1138.7555,0.025090,0.083633,0.062681,0.054930
2,EachMovie,svd,50,128.9592,1138.7555,0.033118,0.099776,0.033355,0.140836
3,EachMovie,svd,100,128.9592,1138.7555,0.036237,0.121420,0.025150,0.210917
4,EachMovie,bpr,10,313.0463,144.5407,0.328606,0.554114,0.367554,0.481217
5,EachMovie,bpr,50,313.0463,144.5407,0.434648,0.636392,0.163551,0.812674
6,EachMovie,bpr,100,313.0463,144.5407,0.452428,0.671695,0.099853,0.907610
7,EachMovie,ncf,10,2092.7641,268.9065,0.266758,0.479525,0.321608,0.419286
8,EachMovie,ncf,50,2092.7641,268.9065,0.360688,0.568510,0.149850,0.760426
9,EachMovie,ncf,100,2092.7641,268.9065,0.378100,0.607271,0.093725,0.866889
10,EachMovie,bivae,10,1163.1346,186.9412,0.293878,0.504554,0.335882,0.452689
